# Sentiment Analysis

Let's make some money.

For our training set, we'll use the Rotten Tomatoes reviews from before. We'll start by using a logistic regression model as our classifier.

In [72]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score

In [73]:
# load data
try:
    df = pd.read_csv('data/rt_critics.csv')
except IOError:
    print 'cannot find file'
    
df.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title
0,Derek Adams,fresh,114709,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559,Toy story
1,Richard Corliss,fresh,114709,TIME Magazine,The year's most inventive comedy.,2008-08-31,9559,Toy story
2,David Ansen,fresh,114709,Newsweek,A winning animated feature that has something ...,2008-08-18,9559,Toy story
3,Leonard Klady,fresh,114709,Variety,The film sports a provocative and appealing st...,2008-06-09,9559,Toy story
4,Jonathan Rosenbaum,fresh,114709,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559,Toy story


If you're up for a challenge, ignore the following cells and create the classifier on your own with your favorite model right here.

See how stop words and tf-idf scoring helps or hurts your model.

When you're done with that, skip to 'Next Steps'

In [74]:
# play space for the bold.




In [75]:
# run this cell to examine data

vectorizer = CountVectorizer()

Xcv = vectorizer.fit_transform(df['quote'])

print '%d samples, %d features' % Xcv.shape

14072 samples, 21544 features


In [113]:
# a helper function to train an SVM model and classify the test instances
def classify_svm(xtrain, xtest, ytrain, ytest):
    clf = svm.SVC(kernel='linear')
    clf.fit(xtrain, ytrain) 
    ypredicted = clf.predict(xtest)
    print "Accuracy: %0.2f%%" % (100 * accuracy_score(ytest, ypredicted))

But wait! We have more features than samples. This would ensure overfitting. Let's trim that number down to the top 5000, ranked by the term frequency across all documents.

In [77]:
# run this cell to vectorize our documents

# create vectorizer object
vectorizer = CountVectorizer(max_features=5000)

# convert our documents and their labels into numpy arrays
Xcv = vectorizer.fit_transform(df['quote'])
Y = (df['fresh'] == 'fresh').values.astype(np.int8)

# split the converted data into training and test sets
xtrain, xtest, ytrain, ytest = train_test_split(Xcv, Y, random_state=1)

In [80]:
# Evaluate performance of models
classify_svm(xtrain, xtest, ytrain, ytest)

Accuracy: 73.31%


# Stop Words

The performance isn't horrible, but it's not great. Can we improve things by [using stop words](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)? See the linked documentation for how to tell CountVectorizer to skip stop words.

In [81]:
# edit this cell to include stopwords

# create vectorizer object
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

# convert our documents and their labels into numpy arrays
Xcvs = vectorizer.fit_transform(df['quote'])
Y = (df['fresh'] == 'fresh').values.astype(np.int8)

# split the converted data into training and test sets
xtraincvs, xtestcvs, ytraincvs, ytestcvs = train_test_split(Xcvs, Y, random_state=1)

In [83]:
# Evaluate performance of models
classify_svm(xtraincvs, xtestcvs, ytraincvs, ytestcvs)

Accuracy: 72.80%


# tf-idf

If that didn't work, how about using tf-idf weighting?

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

In [124]:
# edit this cell to create a TfidfVectorizer instead of a simple CountVectorizer

# create vectorizer object
vectorizer = TfidfVectorizer(max_features=5000)

# convert our documents and their labels into numpy arrays
Xti = vectorizer.fit_transform(df['quote'])
Y = (df['fresh'] == 'fresh').values.astype(np.int8)

# split the converted data into training and test sets
xtrainti, xtestti, ytrainti, ytestti = train_test_split(Xti, Y, random_state=1)

In [125]:
# Evaluate performance of models
classify_svm(xtrainti, xtestti, ytrainti, ytestti)

Accuracy: 76.38%


# tf-idf and stop words

Do both together help?

In [122]:
# edit this cell to create a TfidfVectorizer that uses stop words

# create vectorizer object
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# convert our documents and their labels into numpy arrays
Xtis = vectorizer.fit_transform(df['quote'])
Y = (df['fresh'] == 'fresh').values.astype(np.int8)

# split the converted data into training and test sets
xtraintis, xtesttis, ytraintis, ytesttis = train_test_split(Xtis, Y, random_state=1)

In [123]:
# Evaluate performance of models
classify_svm(xtraintis, xtesttis, ytraintis, ytesttis)

Accuracy: 75.36%


# Next steps

Are you satisfied with these results? Why might you be less than satisfied? How can you explain the observed behavior? What are the next steps you would need to do to improve this classifier? If you have time remaining, try a few strategies out below.

In [13]:
# continue playing here.
# did you finish all of the previous labs? How do your implementations compare?

# More Next Steps

We're not making any money with this classifier yet. If it were that easy, everyone would do it and there'd be no money in it. The hardest part of this problem is usually finding good training data. Googling 'sentiment analysis training data' or 'sentiment analysis test data' turns up a few freely available sources. Most of them are hosted by universities.

But notice, determining the judgment of a movie review isn't the same task as determining the emotional content of a tweet. And yet, it kind of is. The computer doesn't know anything about nature of the text. All it knows is that there are documents with one label (fresh/happy) and documents with another label (rotten/sad) and it needs to fit a model to discriminate between the two. This can be extended to more classes (look into the 20 newsgroups dataset in sci-kit learn) and to proprietary corpora.

One application you might use at work is classifying support emails from users. The classes may be 'ranting', 'mischarge', 'lost order', 'gushing'. Or whatever is common. Even if the classifier isn't perfect, it could help streamline the process of getting the right emails to the right support personnel.